In [1]:
import sys
import os
import cv2 
import numpy as np
import pandas
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras import optimizers
import keras
import pandas as pd
from keras import backend as K

In [2]:
def Conv2d_BN(x, nb_filter, kernel_size, strides=(1,1), padding='same'):
    x = Conv2D(nb_filter, kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization(axis=3)(x)
    x = LeakyReLU(alpha=0.1)(x)
    return x
 
def Conv2dT_BN(x, filters, kernel_size, strides=(2,2), padding='same'):
    x = Conv2DTranspose(filters, kernel_size, strides=strides, padding=padding)(x)
    x = BatchNormalization(axis=3)(x)
    x = LeakyReLU(alpha=0.1)(x)
    return x

In [3]:
def vaeloss(y_true, y_pred):
    # E[log P(X|z)]
    recon = K.sum(K.binary_crossentropy(y_pred, y_true), axis=1)
    #print(type(recon))
    # D_KL(Q(z|X) || P(z|X))
    kl = 0.5 * K.sum(K.exp(z_log_var) + K.square(z_mean) - 1. - z_log_var, axis=1)
    #print(type(kl))
    losses=recon + kl
    return losses
def KL_loss(y_true, y_pred):
	return(0.5 * K.sum(K.exp(z_log_var) + K.square(z_mean) - 1. - z_log_var, axis=1))
def recon_loss(y_true, y_pred):
	return(K.sum(K.binary_crossentropy(y_pred, y_true), axis=1))
def sampling(args):
    z_mean, z_log_var = args
    #print(type(args))
    #print(type(z_log_var))
    #print(type(z_mean))
    epsilon = K.random_normal(shape=(2,), mean=0.,
                              stddev=1.0)
    #print(type(epsilon))
    return z_mean + K.exp(z_log_var / 2) * epsilon

In [13]:
batch_size=6
x = Input(shape=(120,120,3))
conv1 = Conv2d_BN(x, 8, (3, 3))
conv1 = Conv2d_BN(conv1, 8, (3, 3))
pool1 = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(conv1)
conv2 = Conv2d_BN(pool1, 16, (3, 3))
conv2 = Conv2d_BN(conv2, 16, (3, 3))
pool2 = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(conv2)
conv3 = Conv2d_BN(pool2, 32, (3, 3))
conv3 = Conv2d_BN(conv3, 32, (3, 3))
pool3 = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(conv3)
conv4 = Conv2d_BN(pool3, 64, (3, 3))
conv4 = Conv2d_BN(conv4, 64, (3, 3))
pool4 = MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(conv4)
conv5 = Conv2d_BN(pool4, 128, (3, 3))
conv5 = Dropout(0.5)(conv5)
conv5 = Conv2d_BN(conv5, 128, (3, 3))
conv5 = Dropout(0.5)(conv5)
F=Flatten()(conv5)
h = Dense(256, activation='relu')(F)
z_mean = Dense(2)(h)
z_log_var = Dense(2)(h)
z = Lambda(sampling,name = 'sampling', output_shape=(2))([z_mean, z_log_var])
decoder_h = Dense(256, activation='relu')
decoder_mean = Dense(256, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)
decoder=Dense(8*8*128,activation='relu')(x_decoded_mean)
decoder=Reshape((8,8,128))(decoder)
decoder = Conv2DTranspose(128, kernel_size=(3,3), strides=2)(decoder)
decoder = Conv2D(filters=128, kernel_size=(3,3), strides=(1,1), activation='sigmoid')(decoder)
convt1 = Conv2dT_BN(decoder, 64, (3, 3))
convt1 = Dropout(0.5)(convt1)
conv6 = Conv2d_BN(convt1, 64, (3, 3))
conv6 = Conv2d_BN(conv6, 64, (3, 3))
convt2 = Conv2dT_BN(conv6, 32, (3, 3))
convt2  = Dropout(0.5)(convt2)
conv7 = Conv2d_BN(convt2 , 32, (3, 3))
conv7 = Conv2d_BN(conv7, 32, (3, 3))
convt3 = Conv2dT_BN(conv7, 16, (3, 3))
convt3 = Dropout(0.5)(convt3)
conv8 = Conv2d_BN(convt3, 16, (3, 3))
conv8 = Conv2d_BN(conv8, 16, (3, 3))
outpt = Conv2D(filters=3, kernel_size=(1,1), strides=(1,1), padding='same', activation='sigmoid')(conv8)
vae = Model(inputs=x, outputs=outpt)
xent_loss = K.sum(K.binary_crossentropy(x, outpt), axis=-1)
kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(xent_loss + kl_loss)
vae.add_loss(vae_loss)
vae.compile(optimizer='adam',metrics=[KL_loss,recon_loss])
#vae.compile(optimizer='adam',loss=vaeloss,metrics = [KL_loss, recon_loss])
vae.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 120, 120, 3) 0                                            
__________________________________________________________________________________________________
conv2d_72 (Conv2D)              (None, 120, 120, 8)  224         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_76 (BatchNo (None, 120, 120, 8)  32          conv2d_72[0][0]                  
__________________________________________________________________________________________________
leaky_re_lu_76 (LeakyReLU)      (None, 120, 120, 8)  0           batch_normalization_76[0][0]     
____________________________________________________________________________________________

In [6]:
filelist=os.listdir("./dataset/CUB_200_2011/images/")
print(len(filelist))

200


In [7]:
df =  pd.read_csv("./dataset/CUB_200_2011/images.txt",header=None,sep=' ')
#print(df[1].tolist())
filename=df[1].tolist()
print(len(filename))

11788


In [8]:
class generator(keras.utils.Sequence):
    def __init__(self, image_filenames, batch_size):
        self.image_filenames = image_filenames
        self.batch_size = batch_size
    def __len__(self):
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
    def __getitem__(self,idx):
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        train=[]
        labels=[]
        for i in batch_x:
            img=cv2.imread("./dataset/CUB_200_2011/images/"+i)
            img=cv2.resize(img,(120,120))
            img=img/255
            train.append(img)
            labels.append(img)
            n=0
        train=np.array(train)
        labels=np.array(labels)
        #print(train.shape)
        ##print(train_i.shape)
        #print(labels.shape)
        return train,labels

In [9]:
train=[]
test=[]
n=0
for i in filename:
    if(n%10!=0 and n%9!=0):
        train.append(i)
    else:
        test.append(i)
    n=n+1
print(len(train))
print(len(test))
train_load=generator(train,batch_size)
test_load=generator(test,batch_size)

9430
2358


In [14]:
def scheduler(epoch):
    epochs=50
    lr_epochs=200
    lr = K.get_value(vae.optimizer.lr)
    K.set_value(vae.optimizer.lr, lr * (0.1 ** (epoch // lr_epochs)))
    return K.get_value(vae.optimizer.lr)
lr_new = LearningRateScheduler(scheduler)
#x=vae.layers[0].input_shape
#print(x)
from tensorflow.compat.v1 import disable_eager_execution
disable_eager_execution()
vae.call = tf.function(vae.call)
history = vae.fit(train_load,epochs = 50, shuffle=True,verbose = 1,validation_data=test_load,callbacks = [lr_new])
vae.save("./dataset/lmn.h5")

Epoch 1/50


ValueError: ('Error when checking model target: expected no data, but got:', array([[[[0.31764706, 0.46666667, 0.45490196],
         [0.43137255, 0.49803922, 0.51764706],
         [0.49019608, 0.50980392, 0.54509804],
         ...,
         [0.07058824, 0.72941176, 0.56078431],
         [0.04705882, 0.73333333, 0.55686275],
         [0.05490196, 0.71372549, 0.52941176]],

        [[0.2627451 , 0.41568627, 0.39607843],
         [0.43529412, 0.50588235, 0.51372549],
         [0.49411765, 0.52941176, 0.55686275],
         ...,
         [0.08627451, 0.69803922, 0.5372549 ],
         [0.09803922, 0.72941176, 0.55294118],
         [0.09411765, 0.70196078, 0.5254902 ]],

        [[0.23529412, 0.40784314, 0.37647059],
         [0.39215686, 0.47843137, 0.48235294],
         [0.47843137, 0.52156863, 0.5372549 ],
         ...,
         [0.05490196, 0.6745098 , 0.51764706],
         [0.05098039, 0.69019608, 0.51372549],
         [0.05882353, 0.68235294, 0.50196078]],

        ...,

        [[0.109